In [2]:
from dotenv import load_dotenv
from typing import List, Optional, TypedDict
import uuid

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [3]:
class Anime(BaseModel):
    """
    Information about an Anime that the user likes
    """
    
    description: Optional[str] = Field(default=None, description="The description of the anime")

class Data(BaseModel):
    """
    Extracted data about anime into a list of 2 element
    """

    list: List[Anime]

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "You only extract a maximum of 2 elements in the list"
            "You will need to extract a very short description of the anime without giving its name."
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value."
            ,
        ),
        MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [5]:
llm = ChatOpenAI(temperature=1, model="gpt-3.5-turbo-0613", max_tokens=256)
runnable = prompt | llm.with_structured_output(schema=Data)

D:\Anaconda\envs\ml\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The method `ChatOpenAI.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [6]:
text = "I want a romantic anime for girls"
runnable.invoke({"text": text, "examples": [HumanMessage(content="this is an example")]})

Data(list=[Anime(description='A heartwarming story about a girl who falls in love with a boy from her school. They go through various challenges and obstacles as they navigate their feelings for each other.'), Anime(description='A tale of forbidden love between two people from rival families. Despite the obstacles, they are unable to deny their strong feelings for each other.')])

### Adding Reference Examples

In [7]:
class Example(TypedDict):
    """A representation of an example consisting of text input and expected tool calls.

    For extraction, the tool calls are represented as instances of pydantic model.
    """

    input: str  # This is the example text
    tool_calls: List[BaseModel]  # Instances of pydantic model that should be extracted

def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    """Convert an example into a list of messages that can be fed into an LLM.

    This code is an adapter that converts our example to a list of messages
    that can be fed into a chat model.

    The list of messages per example corresponds to:

    1) HumanMessage: contains the content from which content should be extracted.
    2) AIMessage: contains the extracted information from the model
    3) ToolMessage: contains confirmation to the model that the model requested a tool correctly.

    The ToolMessage is required because some of the chat models are hyper-optimized for agents
    rather than for an extraction use case.
    """
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    # The name of the function right now corresponds
                    # to the name of the pydantic model
                    # This is implicit in the API right now,
                    # and will be improved over time.
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages

In [8]:
# Defining examples
examples = [
    (
        "I want an anime about camping.",
        Data(list=[
            Anime(description='A group of high school students go on a camping trip and learn about nature and friendship.'), 
            Anime(description='A girl goes on a solo camping trip to find solace and discover herself in the great outdoors.')
        ])
    ),
    (
        "I want a romantic anime for girls.",
        Data(list=[
            Anime(description='A heartwarming story about two high school students who fall in love and navigate the ups and downs of their relationship.'), 
            Anime(description='A girl moves to a new town and finds herself drawn to a mysterious boy with a tragic past. Together, they discover the power of love and healing.')
        ])
    ),
    (
        "What is 2+2?",
        Data(list=[])
    ),
    (
        "Give me a fun fact",
        Data(list=[])
    ),
    
]

In [9]:
messages = []
for text, tool_call in examples:
    messages.extend(
        tool_example_to_messages({"input": text, "tool_calls": [tool_call]})
    )

In [10]:
prompt.invoke({"text": "What is 2+2?", "examples": messages})

ChatPromptValue(messages=[SystemMessage(content="You are an expert extraction algorithm. You only extract a maximum of 2 elements in the listYou will need to extract a very short description of the anime without giving its name.Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value."), HumanMessage(content='I want an anime about camping.'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'd3932476-b1b4-4003-9040-25843e5f8022', 'type': 'function', 'function': {'name': 'Data', 'arguments': '{"list": [{"description": "A group of high school students go on a camping trip and learn about nature and friendship."}, {"description": "A girl goes on a solo camping trip to find solace and discover herself in the great outdoors."}]}'}}]}, tool_calls=[{'name': 'Data', 'args': {'list': [{'description': 'A group of high school students go on a camping trip and learn about nature and friendshi

In [11]:
text = "an story about war and combat"
data = runnable.invoke({"text": text, "examples": messages})

In [12]:
data

Data(list=[Anime(description='Set in a dystopian future, a group of soldiers fight against an oppressive regime to regain freedom and restore peace to their war-torn land.'), Anime(description='During a devastating war, a young soldier struggles to reconcile his duty with his desire for peace, leading him on a journey of self-discovery and redemption.')])

In [164]:
anime_description = [anime.description for anime in data.list]

In [165]:
anime_description

['In a war-torn world, a group of skilled soldiers must band together to protect their homeland and fight against an oppressive regime.',
 'Set during a fictional war, this story follows a young soldier as he is drafted into battle and faces the harsh realities of combat.']